In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from importlib import reload
import matplotlib.pyplot as plt
reload(plt)


In [ ]:

print(type(sns))
print(type(plt))
print(type(plt.title))


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv("german_credit_data.csv")

In [ ]:
df.head()

In [ ]:
df["Age"].describe()

In the checking account columns we can see a number of rows are NAN. So keeping it or replacing it with some sther value is of no meaning, so we will drop that.

In Risk column Good means low risk and high bad means high risk

In [ ]:
df['Risk'].value_counts()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe(include ="all").T

In [ ]:
df["Job"].unique()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.drop(columns = "Unnamed: 0", inplace = True)

In [ ]:
df.columns

In [ ]:
df[["Age","Credit amount","Duration"]].hist(bins = 10, edgecolor = "black")
plt.suptitle("distribution of Numerical Feature", fontsize = 14)
plt.show()

In [ ]:
plt.figure(figsize= (10,5))
for i , col in enumerate (["Age","Credit amount","Duration"]):
    plt.subplot(1, 3,i + 1)
    sns.boxplot(y = df[col], color = "skyblue")
    plt.title(col)
plt.tight_layout
plt.show()

In [ ]:
df.query("Duration >= 60")

In [ ]:
categorical_cols = ["Sex","Job","Housing","Saving accounts","Checking account","Purpose"]

In [ ]:
plt.figure(figsize = (5,10))
for i , col in enumerate(categorical_cols):
    plt.subplot(3,3,i+1)
    sns.countplot(data = df, x = col, palette = "Set2", order = df[col].value_counts().index)
    plt.title(f"Distribution of {col}")
    plt.xticks(rotation=45)
plt.tight_layout
plt.show()

In [ ]:
corr = df[["Age","Job","Credit amount","Duration"]].corr()

In [ ]:
corr

In [ ]:
sns.heatmap(corr, annot = True, cmap = "coolwarm", fmt = ".2f")
plt.show()

In [ ]:
df.groupby("Job")["Credit amount"].mean()

In [ ]:
df.groupby("Sex")["Credit amount"].mean()

In [ ]:
pd.pivot_table(df, values = "Credit amount",index = "Housing", columns = "Purpose")

In [ ]:
sns.scatterplot(data = df, x = "Age", y = "Credit amount",hue = "Sex", size = "Duration")
plt.title("Credit amount VS Age colored by Sex and sized by Duration")
plt.show()

In [ ]:
sns.violinplot(data = df, x = "Saving accounts", y = "Credit amount",palette="pastel")
plt.title = "Credit amount Vs Saving account"
plt.show()

In [ ]:
df["Risk"].value_counts(normalize = True) * 100

In [ ]:
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.title = pyplot.title
print(type(plt.title))


In [ ]:
from importlib import reload
import matplotlib.pyplot as plt
reload(plt)

In [ ]:
plt.figure(figsize=(15, 4))

for i, col in enumerate(["Age", "Credit amount", "Duration"]):
    plt.subplot(1, 3, i + 1)
    sns.boxplot(data=df, x="Risk", y=col, palette="Pastel2")
    plt.title(f"{col} by Risk")

plt.tight_layout()
plt.show()

In [ ]:
df.groupby("Risk")[["Age","Credit amount","Duration"]].mean()

In [ ]:
categorical_cols

In [ ]:
print(type(plt.title))

In [ ]:
from importlib import reload
import matplotlib.pyplot as plt
reload(plt)

In [ ]:
plt.figure(figsize = (10,10))
for i , col in enumerate(categorical_cols):
    plt.subplot(3, 3, i + 1)
    sns.countplot(data = df, x = col, hue = "Risk", palette = "Set1", order = df[col].value_counts().index)
    plt.title(f"{col} by Risk")
    plt.xticks(rotation = 45)
plt.tight_layout()
plt.show()

In [ ]:
df.columns

In [ ]:
features = ["Age","Sex","Job","Housing","Saving accounts","Checking account","Credit amount","Duration"]

In [ ]:
target = "Risk"

In [ ]:
df_model = df[features + [target]].copy()

In [ ]:
df_model.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib

In [ ]:
cat_cols = df_model.select_dtypes(include = "object").columns.drop("Risk")


In [ ]:
le_dict = {}

In [ ]:
cat_cols

In [ ]:
for col in cat_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    le_dict[col] = le
    joblib.dump(le, f"{col}_encoder.pkl")

In [ ]:
le_target = LabelEncoder()

In [ ]:
target

In [ ]:
df_model[target] = le_target.fit_transform(df_model[target])

In [ ]:
df_model[target]

In [ ]:
joblib.dump(le_target, "target_encoder.pkl")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_model.drop(target, axis = 1)

In [ ]:
y = df_model[target]

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=1)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
def train_model(model, param_grid, X_train, y_train, X_test, y_test):
    grid = GridSearchCV(model, param_grid, cv = 5, scoring = "accuracy", n_jobs = -1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return best_model, acc, grid.best_params_


In [ ]:
dt = DecisionTreeClassifier(random_state = 1, class_weight="balanced")
dt_param_grid = {
    "max_depth" : [3,5,7,10,None],
    "min_samples_split" : [2,5,10],
    "min_samples_leaf" : [1,2,4]
}

In [ ]:
best_dt, acc_dt, params_dt = train_model(dt, dt_param_grid, X_train, y_train, X_test, y_test)

In [ ]:
print("Decision Tree Accurcay", acc_dt)

In [ ]:
print("Best parameters", params_dt)

In [ ]:
rf = RandomForestClassifier(random_state = 1, class_weight="balanced", n_jobs = -1)

In [ ]:
rf_param_grid = {
    "n_estimators": [100,200],
    "max_depth" : [5,7,10,None],
    "min_samples_split" : [2,5,10],
    "min_samples_leaf" : [1,2,4]
}

In [ ]:
best_rf, acc_rf, params_rf = train_model(rf, rf_param_grid, X_train, y_train, X_test, y_test)

In [ ]:
print("Random Forest Accuracy", acc_rf)

In [ ]:
print("Best params", params_rf)

In [ ]:
et = ExtraTreesClassifier(random_state=1, class_weight="balanced",n_jobs=-1)

In [ ]:
et_param_grid = {
    "n_estimators": [100,200],
    "max_depth" : [5,7,10,None],
    "min_samples_split" : [2,5,10],
    "min_samples_leaf" : [1,2,4]
}

In [ ]:
best_et, acc_et, param_et = train_model(et, et_param_grid,X_train,y_train, X_test, y_test)

In [ ]:
print("Extra Tree Accuracy", acc_et)

In [ ]:
print("Best Parameters",param_et)

In [ ]:
xgb = XGBClassifier(random_state = 1, scale_pos_weight = (y_train == 0).sum()/(y_train == 1).sum(), use_LabelEncoder = False, eval_metric = "logloss")

In [ ]:
xgb_param_grid = {
    "n_estimators":[100,200],
    "max_depth" : [3,5,7],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample" : [0.7, 1],
    "colsample_bytree" : [0.7, 1]
}

In [ ]:
best_xgb, acc_xgb, param_xgb = train_model(xgb, xgb_param_grid,X_train,y_train, X_test, y_test)

In [ ]:
print("XGB accuracy",acc_xgb)

In [ ]:
print("Best_params",param_xgb)

In [ ]:
best_et.predict(X_test)

In [ ]:
joblib.dump(best_et, "extra_trees_credit_model.pkl")